In [1]:
import torch
import torch.nn as nn

import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier


In [2]:
torch.cuda.is_available()

True

In [3]:
enr_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/spkvecs.npy')
enr_len = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/enroll/veclabs.npy')


In [4]:
end_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/uttvecs.npy')
ned_vec = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/uttvecs.npy')

save_notin = ned_vec.copy()
np.random.shuffle(save_notin)
np.save('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/shuf_uttvecs.npy', save_notin)

copyd_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/enrollled/test/answer.npy')
ned_cls = np.load('/home/yangwenhao/local/project/SpeakerVerification-pytorch/Data/xvector/MultiResNet10/army/spect_81/soft_nan/x_vector/notenrollled/answer.npy')+51

In [5]:
# Train set
enr_vec_t = enr_vec.transpose()
print("The shape of enrollment vectors are ", enr_vec_t.shape)

# avg_enr_vec = [] #avg vectors for centers
enr_lab = []
for i in range(1, len(enr_len)):
#     avg_enr_vec.append(enr_cos_t[enr_len[i-1]:enr_len[i],:].mean(axis=0).reshape(1,-1))
    for x in range(enr_len[i]-enr_len[i-1]):
        enr_lab.append(i-1)
print("The shape of enrolled vectors' labels :", enr_lab.__len__())

The shape of enrollment vectors are  (1392, 128)
The shape of enrolled vectors' labels : 1392


In [6]:
# Test sets
end_vec_t = end_vec.transpose()
print("The shape of enrolled vectors are ", end_vec.shape)

ned_vec_t = ned_vec.transpose()
print("The shape of not enrolled vectors are ", ned_vec.shape)


The shape of enrolled vectors are  (31487, 128)
The shape of not enrolled vectors are  (11699, 128)


In [7]:
# avg_center = np.concatenate(avg_enr_vec, axis=0).transpose()
# print(avg_center.shape)


In [33]:
# print(ned_cls.shape)
# enr_cos = np.matmul(enr_cos_t, avg_center)

# train 
# data and label
not_enroll_utts = 1392 * 1.2
part_num = int(len(ned_vec)/not_enroll_utts)
train_col = []
test_col = []

for i in range(len(ned_vec)):
    if i%part_num == 2 :
        train_col.append(i)
    else:
        test_col.append(i)
    
    
train_ned = np.delete(ned_vec, test_col, axis=0)
test_ned = np.delete(ned_vec, train_col, axis=0)

x = np.append(enr_vec_t, train_ned, axis=0)
num_noise = 0
if num_noise>0:
    radon_noise = np.random.randn(num_noise, 128)
    x = np.append(x, radon_noise, axis=0)
    
y = np.append(np.ones(len(enr_lab)), np.zeros(len(train_ned)))
y_spk = torch.tensor(enr_lab).long().cuda()
# no out set vectors
# x = enr_vec_t
# y = np.array(enr_lab)-1 #, ned_cls[:100])


In [36]:
# test 
# data and label
# part_ned = np.append(ned_vec[:1000, :], ned_vec[2400:, :], axis=0)
pre_x = np.append(end_vec, test_ned, axis=0)
pre_xz = test_ned

print(pre_x.shape)

real_y = np.append(np.ones(len(end_vec)), np.zeros(len(test_ned)))
# real_yx = np.zeros(len(test_ned))

# print(real_yz.shape, pre_xz.shape)


real_y_spk = y_spk
print(real_y.shape, real_y_spk.shape)

# pre_x = end_vec #, ned_vec[100:], axis=0)
# print(pre_x.shape)
# real_y = end_cls#, ned_cls[100:])
# print(real_y.shape)


(41515, 128)
(41515,) torch.Size([1392])


In [37]:
xlf = DecisionTreeClassifier(random_state=0)
x = x / np.sqrt(np.power(x, 2).sum(axis=1)).reshape(-1,1) * 12
xlf.fit(x, y)

pre_x = pre_x / np.sqrt(np.power(pre_x, 2).sum(axis=1)).reshape(-1,1) * 12
pred_y = xlf.predict(pre_x)

correct = 0
for m,n in zip(pred_y, real_y):
    if m==n:
        correct +=1
acc = correct / len(real_y)
print(acc *100, "%") 
#cos: 
#vec: 71.8493.0256 

62.74358665542575 %


In [54]:
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR,CosineAnnealingLR
import torch.nn.functional as F

linearpred = nn.Sequential(nn.Linear(128, 128), # 使用cos分数预测
                           nn.BatchNorm1d(128),
                           nn.ReLU6(),
                           nn.Dropout(0.1),
                           nn.Linear(128, 1),
                           nn.Sigmoid())

linear_spk = nn.Linear(128, 50)
# linearpred = nn.Linear(128, 2)
bce = nn.BCELoss()
output_softmax = nn.Softmax(dim=1)
crossentropy = nn.CrossEntropyLoss()

optimizer = optim.SGD([{'params': linearpred.parameters()}, {'params': linear_spk.parameters()}], lr=0.1,
                momentum=0.9,
                dampening=0,
                weight_decay=0.0005)

scheduler = MultiStepLR(optimizer, milestones=[80, 120], gamma=0.1)
# scheduler = CosineAnnealingLR(optimizer, T_max = 50)



In [55]:
epoch = 150
t_x = torch.tensor(x).float().cuda()
t_x = F.normalize(t_x, dim=1)*12.0

print(t_x.shape)
linearpred = linearpred.cuda()
linear_spk = linear_spk.cuda()
# t_y = torch.tensor(y).long().cuda()
t_y = torch.tensor(y).float().cuda().squeeze()

print(t_y.shape, t_y.max(),t_y.min())

for i in range(epoch):
#     p_y = linearpred(t_x)
#     loss = crossentropy(p_y, t_y)
#     p_y_lab = output_softmax(p_y)
#     pred_one_labels = torch.max(p_y_lab, dim=1)[1]
    
    spk_l = linear_spk(t_x)
#     print(spk_l.shape)
#     p_y = linearpred(torch.cat((t_x, spk_l), dim=1))# * spk_l.max(dim=1).values.unsqueeze(1))
    p_y = linearpred(t_x)# * spk_l.max(dim=1).values.unsqueeze(1))
    
#     print(spk_l.shape)
    spk_loss = crossentropy(spk_l[:len(y_spk), :], y_spk)
    spk_lab = output_softmax(spk_l)
    pred_one_labels = torch.max(spk_lab[:len(y_spk), :], dim=1)[1]
    
    corrects = float((pred_one_labels.cuda() == y_spk).sum().item())
    accuracy_spk = corrects / len(y_spk)
    
    loss = spk_loss + bce(p_y.squeeze(), t_y) #+
#     print(p_y.shape)
    pred_one_labels = torch.where(p_y.squeeze().cpu() > 0.5, torch.ones(len(p_y)), torch.zeros(len(p_y)))
    corrects = float((pred_one_labels.cuda() == t_y).sum().item())
    accuracy_set = corrects / len(p_y)
    
    if (i+1) % 5 == 0:
        print("The avg_prob:  For inset spk: %.4f " % float(torch.max(spk_lab[:len(y_spk), :], dim=1)[0].mean()), end="")
        print(" outset spk: %.4f " % float(torch.max(spk_lab[len(y_spk):, :], dim=1)[0].mean()))
        print(" Loss: %.5f Accuracy spk: %.4f %% set: %.4f %% " % (loss.item(), accuracy_spk*100., accuracy_set*100.)) # 100%
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

torch.Size([3063, 128])
torch.Size([3063]) tensor(1., device='cuda:0') tensor(0., device='cuda:0')

The avg_prob:  For inset spk: 0.1263  outset spk: 0.0995 
 Loss: 3.10288 Accuracy spk: 59.4109 % set: 67.4502 % 

The avg_prob:  For inset spk: 0.6840  outset spk: 0.5521 
 Loss: 1.07803 Accuracy spk: 93.8218 % set: 76.9833 % 

The avg_prob:  For inset spk: 0.9001  outset spk: 0.7619 
 Loss: 0.56431 Accuracy spk: 98.4195 % set: 82.1417 % 

The avg_prob:  For inset spk: 0.9600  outset spk: 0.8179 
 Loss: 0.41321 Accuracy spk: 99.2816 % set: 83.5129 % 

The avg_prob:  For inset spk: 0.9773  outset spk: 0.8396 
 Loss: 0.33816 Accuracy spk: 99.6408 % set: 86.3206 % 

The avg_prob:  For inset spk: 0.9839  outset spk: 0.8493 
 Loss: 0.28431 Accuracy spk: 99.7126 % set: 89.0957 % 

The avg_prob:  For inset spk: 0.9871  outset spk: 0.8539 
 Loss: 0.25059 Accuracy spk: 99.7845 % set: 90.7607 % 

The avg_prob:  For inset spk: 0.9889  outset spk: 0.8560 
 Loss: 0.22063 Accuracy spk: 99.9282 % set: 

In [56]:
lin_pre_x = torch.tensor(pre_x).cuda()
lin_pre_x = F.normalize(lin_pre_x, dim=1)*12.0

# test 
# data and label
# part_ned = np.append(ned_vec[:1000, :], ned_vec[2400:, :], axis=0)

# outset samples and labels
neg_pre_x = np.array(test_ned)
print(pre_x.shape)
neg_real_y = np.zeros(len(test_ned))
print(real_y.shape)

neg_pre_x = torch.tensor(neg_pre_x).cuda()
neg_pre_x = F.normalize(neg_pre_x, dim=1)*12.0

linearpred.eval()
with torch.no_grad():
#     lin_pre_y = linearpred(lin_pre_x)
#     lin_p_y_lab = output_softmax(lin_pre_y)
#     lin_pred_labels = torch.max(lin_p_y_lab, dim=1)[1]
    lin_real_y = torch.tensor(real_y).long().cuda()
    spk_l = linear_spk(lin_pre_x)
    neg_real_y = torch.tensor(neg_real_y).long().cuda()
    
#     print(spk_l.shape)
    for t in range(1, 100):
        inout_t = t * 0.01
#         lin_pre_y = linearpred(torch.cat((lin_pre_x, spk_l), dim=1))
        lin_pre_y = linearpred(lin_pre_x)
        pred_one_labels = torch.where(lin_pre_y.squeeze().cpu()>inout_t, torch.ones(len(real_y)), torch.zeros(len(real_y)))
        corrects = float((pred_one_labels.long().cuda() == lin_real_y).sum().item())
        lin_test_accuracy = corrects / len(lin_real_y)
        print("t = %.2f, inset Acc: %.2f mean_val: %.4f"%(inout_t, lin_test_accuracy*100., lin_pre_y.cpu().mean()), end=" ") 

#         neg_pre_y = linearpred(torch.cat((neg_pre_x, linear_spk(neg_pre_x)), dim=1))
        neg_pre_y = linearpred(neg_pre_x)

        pred_one_labels = torch.where(neg_pre_y.squeeze().cpu()>inout_t, torch.ones(len(neg_real_y)), torch.zeros(len(neg_real_y)))
        corrects = float((pred_one_labels.long().cuda() == neg_real_y).sum().item())
        lin_test_accuracy = corrects / len(neg_real_y)
        print("outset Acc: %.2f, mean_val: %.2f" %( lin_test_accuracy*100., neg_pre_y.cpu().mean())) 

# 26.65% # cos sim include out set
# 71.3301% # vec without out set data 80 epoch
# 74

(41515, 128)
(41515,)
t = 0.01 ,inset predict acc: 79.67 mean_val: 0.5101 outset: 40.69, mean_val: 0.09
t = 0.02 ,inset predict acc: 79.71 mean_val: 0.5101 outset: 53.50, mean_val: 0.09
t = 0.03 ,inset predict acc: 79.52 mean_val: 0.5101 outset: 60.77, mean_val: 0.09
t = 0.04 ,inset predict acc: 79.39 mean_val: 0.5101 outset: 65.53, mean_val: 0.09
t = 0.05 ,inset predict acc: 79.22 mean_val: 0.5101 outset: 69.32, mean_val: 0.09
t = 0.06 ,inset predict acc: 79.02 mean_val: 0.5101 outset: 72.08, mean_val: 0.09
t = 0.07 ,inset predict acc: 78.88 mean_val: 0.5101 outset: 74.46, mean_val: 0.09
t = 0.08 ,inset predict acc: 78.78 mean_val: 0.5101 outset: 76.57, mean_val: 0.09
t = 0.09 ,inset predict acc: 78.61 mean_val: 0.5101 outset: 78.06, mean_val: 0.09
t = 0.10 ,inset predict acc: 78.46 mean_val: 0.5101 outset: 79.37, mean_val: 0.09
t = 0.11 ,inset predict acc: 78.31 mean_val: 0.5101 outset: 80.72, mean_val: 0.09
t = 0.12 ,inset predict acc: 78.10 mean_val: 0.5101 outset: 81.63, mean_val: